In [ ]:
using HTTP
using CodecZlib
using DataFrames
using Mongoc

In [ ]:
function  getDataFromURLFile(url_file)

    i = 0 ;
    while i < 5
        try 
          
            response =  HTTP.request("GET", url_file);
            body = response.body;
            filehandler = GzipDecompressorStream(IOBuffer(body));
            imdb_data =  collect(eachline(filehandler));
            
            return imdb_data;
            
            break
        catch e
            println(e);
            i += 1;
        end
    end

end

In [ ]:
function get_imdb_payload(imdb_data, header)
    imdb_data = Vector{Dict}()  # acá hay que ver si cambiar imdb por objeto BSon de Mongo
    for data in imdb_data
        subdata_imdb = Dict()
        for (key, value) in zip(header, data)
            println(key, value)
            if occursin(r"^[\d(.)?]+$", value)
                value = parse.(Float64,value)
            end
            subdata_imdb[key] = value
            
        end
        # println(subdata_mongo)
        push!(imdb_data_data, subdata_imdb)
        println(imdb_data_data)
        
    end
    
    return mongo_data
end

In [ ]:
function extract_imdb_data(imdb_file)
    file_name = split("https://datasets.imdbws.com/title.ratings.tsv.gz", "/")[end]
    imdb_data  = getDataFromURLFile(imdb_file);
    imdb_data = [split(x, "\t") for x in imdb_data];
    header = imdb_data[1];
    deleteat!(imdb_data, 1);
    imdb_payload = get_imdb_payload(imdb_data, header)
    return imdb_payload
end


function proccess_imdb_files(imdb_files)
    imdb_payloads = Vector{Vector{Dict}}()
    for imdb_file in imdb_files
        imdb_payload = extract_imdb_data(imdb_file)
        push!(imdb_payloads, imdb_payload)
        println(imdb_payloads)
    end
end


In [ ]:
imdb_files = ["https://datasets.imdbws.com/title.ratings.tsv.gz"]